# 1.Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics as mt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
import pickle

# 2. Funcao para carregar os datasets

In [2]:
def Load_Data_Set(df_x_train, df_y_train):
    df1 = pd.read_csv(df_x_train)
    df2 = pd.read_csv(df_y_train)
    return df1, df2

# 3. Criando e concatenando os dataframes

### 3.1 Dataset de treinamento

In [3]:
df_x_train = "../2_ensaio_regressao/1_dados_treinamento/X_training.csv"

In [4]:
df_y_train = "../2_ensaio_regressao/1_dados_treinamento/y_training.csv"

In [5]:
df1_train, df2_train = Load_Data_Set(df_x_train, df_y_train)

In [6]:
df1_train['label'] = df2_train

In [7]:
df_train = df1_train.copy()

### 3.2 Dataset de validacao

In [8]:
df_x_val = "../2_ensaio_regressao/2_dados_validacao/X_validation.csv"

In [9]:
df_y_val = "../2_ensaio_regressao/2_dados_validacao/y_val.csv"

In [10]:
df1_val, df2_val = Load_Data_Set(df_x_val, df_y_val)

In [11]:
df1_val['label'] = df2_val

In [12]:
df_val = df1_val.copy()

### 3.3 Dataset de teste

In [13]:
df_x_test = "../2_ensaio_regressao/3_dados_teste/X_test.csv"

In [14]:
df_y_test = "../2_ensaio_regressao/3_dados_teste/y_test.csv"

In [15]:
df1_test, df2_test = Load_Data_Set(df_x_test, df_y_test)

In [16]:
df1_test['label'] = df2_test

In [17]:
df_test = df1_test.copy()

# 4. Funções para treinar o modelo e avaliar as métricas

In [18]:
#Criando uma função para Treinar o Modelo encima dos dados de treinamento

#Fazer as previsoes encima dos proprios dados de treinamento

#Em seguida, fazer previsoes encima dos dados de teste e validacao

#Entrada: dataframes de treinamento e validação + grau do polinomio + alpha (parametro da Regressao Ridge) + 
#max_iter (parametro da Regressao Ridge) 

#Saida: valores de y_train + valores de previsao feitos a partir dos dados de treinamento (y_pred_train) +
# valores de y_val + valor de previsao de y_val


def Model_Training(df_train, df_val, degree, alpha, max_iter):
    
    features = ['song_duration_ms', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'audio_mode',
       'speechiness', 'tempo', 'time_signature', 'audio_valence']
        
    label = ['label']
    
    #Preparação dos dados de treino
    x_train = df_train.loc[:, features]

    #Esse metodo ravel transforma os valores em um array
    y_train = df_train.loc[:, label].values.ravel()    
    
    #Preparação dos dados de teste
    
    x_test = df_test.loc[:, features]

    y_test = df_test.loc[:, label].values.ravel()  
        
    #Preparação dos dados de validação
    x_val = df_val.loc[:, features]

    y_val = df_val.loc[:, label].values.ravel()
    
    #Treinamento do algoritmo de Regressao polinomial     
        
    # Criando a matriz de features polinomiais
    poly = PolynomialFeatures(degree=degree)
    
    X_poly_train = poly.fit_transform(x_train)
    # Treinando o modelo de regressão polinomial
    model = Ridge(alpha = alpha, max_iter = max_iter)
    
    model = model.fit(X_poly_train, y_train)
  
    #Fazendo previsoes emcima dos dados de treinamento
    y_pred_train = model.predict(X_poly_train)
    
    #Fazendo previsoes encima dos dados de validacao
    X_poly_val = poly.fit_transform(x_val)
    y_pred_val = model.predict(X_poly_val)
    
    return y_train, y_pred_train, y_val, y_pred_val  


In [19]:
#Criando uma função para avaliar as metricas do modelo

#Entrada: Os proprios valores da label daquele dataset, as previsoes feitas a partir desses dados
# os valores de parametro: degree (grau do polinomio) + alpha (Ridge) + max_iter (Ridge)

#Saida: Dataframe com as principais metricas do modelo

def Model_Metrics(y, y_pred, degree, alpha, max_iter):
    
    #R2_Score
    r2_score = np.round(mt.r2_score(y , y_pred ),4)
        
    #Mean_Squared_Error (MSE)
    mean_squared_error = np.round(mt.mean_squared_error(y, y_pred ),4)
        
    #Root Mean_Squared Error (RMSE)
    rmse = np.round(np.sqrt(mean_squared_error),4)
        
    #Mean_Absolute_Error(MAE)
    mean_absolute_error = np.round(mt.mean_absolute_error (y,y_pred),4)
   
    #Mean Absolute Percentage Error (MAPE)
    mean_absolute_percentage_error = np.round(mt.mean_absolute_percentage_error (y,y_pred),4)
    
    #Criando um dataframe com as metricas
    
    parameters = 'degree = ' + str(degree) + ' ;alpha = ' + str(alpha) + ' ;max_iter = ' + str(max_iter)
    
    d = { parameters: [r2_score, mean_squared_error , rmse , mean_absolute_error ,mean_absolute_percentage_error]}  
    
    df_metrics =  pd.DataFrame(data = d, index = ['r2_score', 'mse', 'rmse', 'mae',
                                                 'mape'])  
    
        
    return df_metrics

# 5. Treinamento do modelo, validação e verificação de performance

### 5.1 Treinar o modelo com os valores de parametro default - degree = 2, alpha = 1, max-iter = None e verificação das métricas do modelo sobre os dados de treinamento

In [20]:
y_train, y_pred_train, y_val, y_pred_val = Model_Training(df_train, df_val, 2, 1, None)
df_default_train = Model_Metrics(y_train, y_pred_train, 2,1,None)
df_default_train

,degree = 2 ;alpha = 1 ;max_iter = None
r2_score,0.0932
mse,433.4755
rmse,20.8201
mae,16.4720
mape,8.3727


### 5.2 Testar o algoritmo com o parametro default  nos dados de validação e verificar sua performance

In [21]:
df_default_val = Model_Metrics(y_val, y_pred_val,2,1,None)
df_default_val

,degree = 2 ;alpha = 1 ;max_iter = None
r2_score,0.0677
mse,445.1844
rmse,21.0994
mae,16.7387
mape,8.5690


### 5.3 Treinar o modelo nos dados de treinamento alterando os parametros e testar a performance desse modelo sobre os dados de validação

### a) Degree = 4, alpha = 1, max_iter = None

In [22]:
y_train, y_pred_train, y_val, y_pred_val = Model_Training(df_train, df_val, 4,1, None)
df_val1 = Model_Metrics(y_val, y_pred_val, 4,1, None)
df_val1

,degree = 4 ;alpha = 1 ;max_iter = None
r2_score,-28.4379
mse,14056.9522
rmse,118.5620
mae,18.9702
mape,8.4648


### b) Degree = 6 ; alpha = 1, max_iter = None

In [23]:
y_train, y_pred_train, y_val, y_pred_val = Model_Training(df_train, df_val, 6,1,None)
df_val2 = Model_Metrics(y_val, y_pred_val, 6,1,None)
df_val2

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:194: LinAlgWarning: Ill-conditioned matrix (rcond=8.32883e-18): result may not be accurate.
  dual_coef = linalg.solve(K, y, sym_pos=True, overwrite_a=False)


,degree = 6 ;alpha = 1 ;max_iter = None
r2_score,-4.545593e+05
mse,2.170578e+08
rmse,1.473288e+04
mae,3.270700e+02
mape,1.855360e+01


### c) Degree = 2 , alpha = 1, max_iter = 500

In [24]:
y_train, y_pred_train, y_val, y_pred_val = Model_Training(df_train,df_val, 2,1,500)
df_val3 = Model_Metrics(y_val, y_pred_val, 2,1,500)
df_val3

,degree = 2 ;alpha = 1 ;max_iter = 500
r2_score,0.0677
mse,445.1844
rmse,21.0994
mae,16.7387
mape,8.5690


### d) Degree = 2, alpha = 10, max_iter = 500

In [25]:
y_train, y_pred_train, y_val, y_pred_val = Model_Training(df_train,df_val, 2,10,500)
df_val4 = Model_Metrics(y_val, y_pred_val, 2,10,500)
df_val4

,degree = 2 ;alpha = 10 ;max_iter = 500
r2_score,0.0672
mse,445.4011
rmse,21.1045
mae,16.7444
mape,8.5983


### e) Degree = 2, alpha = 0.5, max_iter = None

In [26]:
y_train, y_pred_train, y_val, y_pred_val = Model_Training(df_train,df_val, 2,0.5,None)
df_val5 = Model_Metrics(y_val, y_pred_val,2,0.5,None)
df_val5

,degree = 2 ;alpha = 0.5 ;max_iter = None
r2_score,0.0677
mse,445.2026
rmse,21.0998
mae,16.7382
mape,8.5638


### f) Degree = 2, alpha = 0.1, max_iter = None

In [27]:
y_train, y_pred_train, y_val, y_pred_val = Model_Training(df_train,df_val, 2,0.1,None)
df_val6 = Model_Metrics(y_val, y_pred_val,2,0.1,None)
df_val6

,degree = 2 ;alpha = 0.1 ;max_iter = None
r2_score,0.0672
mse,445.4171
rmse,21.1049
mae,16.7418
mape,8.5541


### 5.4 Verificar o parametro que obteve os melhores resultados nos dados de validação

In [28]:
df_metrics_val = pd.concat([df_default_val, df_val1, df_val2, df_val3, df_val4, df_val5,df_val6
                           ] , axis = 1)
df_metrics_val

,degree = 2 ;alpha = 1 ;max_iter = None,degree = 4 ;alpha = 1 ;max_iter = None,degree = 6 ;alpha = 1 ;max_iter = None,degree = 2 ;alpha = 1 ;max_iter = 500,degree = 2 ;alpha = 10 ;max_iter = 500,degree = 2 ;alpha = 0.5 ;max_iter = None,degree = 2 ;alpha = 0.1 ;max_iter = None
r2_score,0.0677,-28.4379,-4.545593e+05,0.0677,0.0672,0.0677,0.0672
mse,445.1844,14056.9522,2.170578e+08,445.1844,445.4011,445.2026,445.4171
rmse,21.0994,118.5620,1.473288e+04,21.0994,21.1045,21.0998,21.1049
mae,16.7387,18.9702,3.270700e+02,16.7387,16.7444,16.7382,16.7418
mape,8.5690,8.4648,1.855360e+01,8.5690,8.5983,8.5638,8.5541


#### A combinação de parâmetros que apresentou o melhor resultado foi a que está alocada no objeto df_val5, a saber degree = 2, alpha = 0.5, max_iter = None

# 6. Unificar os dados de treinamento e validação e retreinar o algoritmo, utilizando os melhores valores para os parâmetros

### 6.1 Dividir novamente os datasets de treinamento e validação

In [29]:
features = ['song_duration_ms', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'audio_mode',
       'speechiness', 'tempo', 'time_signature', 'audio_valence']
    
    
label = ['label'] 

#Separação dos dados de treino
x_train = df_train.loc[:, features]

#Esse metodo ravel transforma os valores em um array
y_train = df_train.loc[:, label].values.ravel()      

#Separação dos dados de validação
x_val = df_val.loc[:, features]

y_val = df_val.loc[:, label].values.ravel() 

#Separação dos dados de teste
x_test = df_test.loc[:, features]

y_test =  df_test.loc[:, label].values.ravel() 


#Unificando as features e a label dos dados de treino com as dos dados de validação

x_final = np.concatenate( (x_train, x_val) )

y_final = np.concatenate((y_train, y_val))

### 6.2 Treinar o modelo novamente com o melhor parametro obtido e fazer previsões encima dos dados de teste

#### Os polinomios de graus maiores certamente provocariam overfitting nos dados. Optei por manter o treinamento final com polinomio grau 2

In [30]:
# Criando a matriz de features polinomiais
poly = PolynomialFeatures(degree=2)
    
X_poly_final = poly.fit_transform(x_final)
# Treinando o modelo de regressão polinomial
model_last = Ridge(alpha = 0.5, max_iter=None)    
model_last = model_last.fit(X_poly_final, y_final)


#Transformando e fazendo previsões encima dos dados de teste
X_poly_test = poly.fit_transform(x_test)
y_pred_test = model_last.predict(X_poly_test)


### 6.3 Verificando as métricas do modelo com os dados de teste

In [31]:
df_metrics_test = Model_Metrics(y_test, y_pred_test, 2, 0.5, None)
df_metrics_test

,degree = 2 ;alpha = 0.5 ;max_iter = None
r2_score,0.0906
mse,442.7649
rmse,21.0420
mae,16.7377
mape,8.2971


# 7. Resguardando os objetos com as métricas de maior performance

### 7.1 Alterando o nome das colunas dos dataframes

#### a) dataframe teste com melhores métricas

In [32]:
df_metrics_test = df_metrics_test.rename(columns = {"degree = 2 ;alpha = 0.5 ;max_iter = None": "Polinomial Regression Ridge"})
df_metrics_test

,Polinomial Regression Ridge
r2_score,0.0906
mse,442.7649
rmse,21.0420
mae,16.7377
mape,8.2971


#### b) dataframe validação com melhores métricas

In [33]:
df_val5 = df_val5.rename(columns = {"degree = 2 ;alpha = 0.5 ;max_iter = None": "Polinomial Regression Ridge"})
df_val5

,Polinomial Regression Ridge
r2_score,0.0677
mse,445.2026
rmse,21.0998
mae,16.7382
mape,8.5638


#### c) dataframe treinamento com melhores métricas

In [34]:
df_default_train = df_default_train.rename(columns={"degree = 2 ;alpha = 1 ;max_iter = None": "Polinomial Regression Ridge"})
df_default_train

,Polinomial Regression Ridge
r2_score,0.0932
mse,433.4755
rmse,20.8201
mae,16.4720
mape,8.3727


# 8. Salvar os objetos com as melhores metricas em um arquivo pickle

### a) dataframe com métricas dos dados de treinamento

In [35]:
with open('arquivo_poly_regression_ridge_train.pkl', 'wb') as arquivo_poly_regression_ridge_train:
    pickle.dump(df_default_train,arquivo_poly_regression_ridge_train)  

In [36]:
arquivo_poly_regression_ridge_train.close()

### b) dataframe com métricas dos dados de validação

In [37]:
with open('arquivo_poly_regression_ridge_val.pkl', 'wb') as arquivo_poly_regression_ridge_val:
    pickle.dump(df_val5,arquivo_poly_regression_ridge_val) 

In [38]:
arquivo_poly_regression_ridge_train.close()

### c) dataframe com métricas dos dados de teste

In [39]:
with open('arquivo_poly_regression_ridge_test.pkl', 'wb') as arquivo_poly_regression_ridge_test:
    pickle.dump(df_metrics_test,arquivo_poly_regression_ridge_test) 

In [40]:
arquivo_poly_regression_ridge_test.close()